# 📄 Explore Raw Documents (extended)

Цей ноутбук:
- показує всі файли у `data/raw/`
- рахує базову статистику (символи/рядки/слова)
- перевіряє, чи є структура для BioConsult: `ГРУПА ЗАПИТУ:` / `Ключові слова:` / `Відповідь:`
- знаходить сміття типу `=====` / `-----` і підсвічує, в яких файлах є
- робить прев’ю знайдених груп (назва + ключі + перші рядки відповіді)


In [ ]:
from pathlib import Path
import re
from collections import Counter

RAW_DIR = Path("../data/raw")
print("📂 RAW_DIR:", RAW_DIR.resolve())

def safe_read(p: Path):
    try:
        return p.read_text(encoding="utf-8"), None
    except Exception as e:
        return None, e

def basic_stats(text: str):
    lines = text.splitlines()
    words = re.findall(r"\b[\w’']+\b", text, flags=re.UNICODE)
    return {
        "chars": len(text),
        "lines": len(lines),
        "words": len(words),
    }

SEP_RE = re.compile(r"^(={3,}|-{3,}|_{3,}|\*{3,})\s*$")

GROUP_RE = re.compile(r"^ГРУПА\s+ЗАПИТУ:\s*(.+)\s*$", re.IGNORECASE)
KEYS_RE  = re.compile(r"^Ключові\s+слова:\s*(.+)\s*$", re.IGNORECASE)
ANS_RE   = re.compile(r"^Відповідь:\s*$", re.IGNORECASE)

def parse_groups(txt: str):
    """Парсер під формат BioConsult."""
    lines = txt.splitlines()
    groups = []
    cur = None
    mode = None  # None | 'ans'

    for raw in lines:
        line = raw.rstrip("\n")

        mG = GROUP_RE.match(line.strip())
        if mG:
            if cur:
                cur["answer"] = cur["answer"].strip()
                groups.append(cur)
            cur = {"title": mG.group(1).strip(), "keys": [], "answer": ""}
            mode = None
            continue

        if cur is None:
            continue

        mK = KEYS_RE.match(line.strip())
        if mK:
            keys_line = mK.group(1).strip()
            keys = [k.strip() for k in keys_line.split(",") if k.strip()]
            cur["keys"] = keys
            continue

        if ANS_RE.match(line.strip()):
            mode = 'ans'
            continue

        if mode == 'ans':
            cur["answer"] += raw + "\n"

    if cur:
        cur["answer"] = cur["answer"].strip()
        groups.append(cur)

    return groups

files = sorted([p for p in RAW_DIR.glob("*") if p.is_file()])
if not files:
    print("⚠️ У data/raw нічого не знайдено")
else:
    print(f"✅ Знайдено файлів: {len(files)}")

report = []
for p in files:
    text, err = safe_read(p)
    item = {"file": p.name, "ok": err is None, "error": str(err) if err else ""}

    if err is None:
        st = basic_stats(text)
        item.update(st)

        # separators (==== etc)
        sep_lines = [i+1 for i, ln in enumerate(text.splitlines()) if SEP_RE.match(ln.strip())]
        item["sep_lines_count"] = len(sep_lines)
        item["sep_lines_sample"] = sep_lines[:10]

        # structure check
        has_g = bool(re.search(r"^ГРУПА\s+ЗАПИТУ:", text, flags=re.IGNORECASE | re.MULTILINE))
        has_k = bool(re.search(r"^Ключові\s+слова:", text, flags=re.IGNORECASE | re.MULTILINE))
        has_a = bool(re.search(r"^Відповідь:", text, flags=re.IGNORECASE | re.MULTILINE))
        item["has_groups_format"] = bool(has_g and has_k and has_a)

        groups = parse_groups(text)
        item["groups_count"] = len(groups)

        # preview first 2 groups
        previews = []
        for g in groups[:2]:
            ans_preview = " ".join(g["answer"].splitlines()[:3])
            ans_preview = re.sub(r"\s+", " ", ans_preview).strip()
            previews.append({
                "title": g["title"],
                "keys": g["keys"],
                "answer_preview": ans_preview[:240] + ("…" if len(ans_preview) > 240 else "")
            })
        item["groups_preview"] = previews

    report.append(item)

print("\n" + "="*70)
for r in report:
    print(f"\n📄 {r['file']}")
    if not r["ok"]:
        print("  ❌ Cannot read:", r["error"])
        continue
    print(f"  chars={r['chars']}  lines={r['lines']}  words={r['words']}")
    print(f"  separators(==== etc): {r['sep_lines_count']}  sample_lines={r['sep_lines_sample']}")
    print(f"  has_groups_format: {r.get('has_groups_format', False)}  groups_count={r.get('groups_count', 0)}")
    if r.get('groups_preview'):
        for i, pv in enumerate(r['groups_preview'], 1):
            print(f"    ├─ Group {i}: {pv['title']}")
            print(f"    │  keys: {', '.join(pv['keys'][:12])}{'…' if len(pv['keys'])>12 else ''}")
            print(f"    │  preview: {pv['answer_preview']}")
    else:
        print("  (no groups preview)")
print("\n" + "="*70)


## ✅ (Опційно) Детальний перегляд конкретного файлу
Введи ім’я файлу (наприклад `biology_basics.txt`) — і подивимось групи та ключові слова.

In [ ]:
TARGET = "biology_basics.txt"  # <-- зміни, якщо треба
p = RAW_DIR / TARGET

text, err = safe_read(p)
if err:
    print("❌ Cannot read", TARGET, ":", err)
else:
    groups = parse_groups(text)
    print(f"✅ {TARGET}: groups={len(groups)}")
    for i, g in enumerate(groups[:10], 1):
        print("\n" + "-"*60)
        print(f"[{i}] {g['title']}")
        print("keys:", ", ".join(g['keys'][:20]) + ("…" if len(g['keys'])>20 else ""))
        ans_lines = [ln for ln in g['answer'].splitlines() if ln.strip()]
        print("answer first lines:")
        for ln in ans_lines[:6]:
            print(" ", ln)


## 🧹 (Опційно) Перевірка «сміття» у відповідях
Це корисно, якщо в базі попадаються лінії типу `=====` і вони потім з’являються у відповіді бота.

In [ ]:
TARGET = "biology_basics.txt"
p = RAW_DIR / TARGET
text, err = safe_read(p)
if err:
    print("❌ Cannot read", TARGET, ":", err)
else:
    lines = text.splitlines()
    bad = [(i+1, ln) for i, ln in enumerate(lines) if SEP_RE.match(ln.strip())]
    print(f"✅ Found separator-lines: {len(bad)}")
    for i, (lnum, ln) in enumerate(bad[:30], 1):
        print(f"{i:02d}. line {lnum}: {ln}")
    if len(bad) > 30:
        print("…")
